In [1]:
import mediapipe as mp
import cv2
from matplotlib import pyplot as plt

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def rozpoznawanie(obraz, model):
    obraz = cv2.cvtColor(obraz, cv2.COLOR_BGR2RGB)
    obraz.flags.writeable = False
    wynik = model.process(obraz)
    obraz.flags.writeable = True                   
    obraz = cv2.cvtColor(obraz, cv2.COLOR_RGB2BGR)
    return obraz, wynik

In [4]:
def szkielet(obraz, wyniki):
    mp_drawing.draw_landmarks(obraz, wyniki.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(obraz, wyniki.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
           

In [5]:
import os
import numpy as np
sciezka = os.path.join('dane')  

gesty = np.array(['czesc','dziekuje'])

liczba_sekwencji = 30

dlugosc_sekwencji = 30

In [6]:
def zwroc_punkty(wyniki):
    lewa = np.zeros(21*3)
    prawa = np.zeros(21*3)

    if wyniki.left_hand_landmarks:
        lewa = np.array([[res.x, res.y, res.z] for res in wyniki.left_hand_landmarks.landmark]).flatten()

    if wyniki.right_hand_landmarks:
        prawa = np.array([[res.x, res.y, res.z] for res in wyniki.right_hand_landmarks.landmark]).flatten()

    return np.concatenate([ lewa, prawa])

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [8]:
nazwy_mapa = {nazwa:num for num, nazwa in enumerate(gesty)}

In [9]:
nazwy_mapa

{'czesc': 0, 'dziekuje': 1}

In [10]:
sekwencje, nazwy = [], []
for akcja in gesty:
    for sekwencja in range(liczba_sekwencji):
        okno = []
        for nr_klatki in range(dlugosc_sekwencji):
            res = np.load(os.path.join(sciezka, akcja, str(sekwencja), "{}.npy".format(nr_klatki)))
            okno.append(res)
        sekwencje.append(okno)
        nazwy.append(nazwy_mapa[akcja])

In [11]:
X = np.array(sekwencje)
y = to_categorical(nazwy).astype(int)

In [12]:
X.shape

(60, 30, 126)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,
                                              stratify=y,random_state=1 )
X_valid,X_test,y_valid,y_test=train_test_split(X_test,y_test,test_size=0.5,
                                              stratify=y_test,random_state=1 )

In [15]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from tensorflow.keras import optimizers
import tensorflow as tf

In [16]:
def stworz_model(X_train, y_train, X_test, y_test):
	l_krokow, l_featerow, l_outputow = X_train.shape[1], X_train.shape[2], y_train.shape[1]
	model = Sequential()
	model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(l_krokow,l_featerow)))
	model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
	model.add(Dropout(0.5))
	model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
	model.add(tf.keras.layers.Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(l_outputow, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [17]:
model=stworz_model(X_train,y_train, X_test,  y_test)

In [18]:
history = model.fit(X_train,y_train,
                  validation_data=(X_valid,y_valid),
                  batch_size=128,
                  epochs=20
)

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.7587 - accuracy: 0.4444 - val_loss: 0.4421 - val_accuracy: 0.6667
Epoch 2/20
1/1 [==============================] - 0s 64ms/step - loss: 0.4675 - accuracy: 0.5278 - val_loss: 0.3231 - val_accuracy: 1.0000
Epoch 3/20
1/1 [==============================] - 0s 60ms/step - loss: 0.2766 - accuracy: 0.9722 - val_loss: 0.2378 - val_accuracy: 1.0000
Epoch 4/20
1/1 [==============================] - 0s 68ms/step - loss: 0.1655 - accuracy: 1.0000 - val_loss: 0.1597 - val_accuracy: 1.0000
Epoch 5/20
1/1 [==============================] - 0s 70ms/step - loss: 0.0813 - accuracy: 1.0000 - val_loss: 0.0941 - val_accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 0s 62ms/step - loss: 0.0409 - accuracy: 1.0000 - val_loss: 0.0518 - val_accuracy: 1.0000
Epoch 7/20
1/1 [==============================] - 0s 64ms/step - loss: 0.0199 - accuracy: 1.0000 - val_loss: 0.0272 - val_accuracy: 1.0000
Epoch 8/20
1/1 [=============

In [19]:
score,acc=model.evaluate(X_test,y_test)
print('Test score:', score)
print('Test accuracy:', acc)

1/1 [==============================] - 0s 50ms/step - loss: 2.3671e-05 - accuracy: 1.0000
Test score: 2.367142769799102e-05
Test accuracy: 1.0


In [20]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [21]:
kolory = [(245,117,16), (117,245,16), (16,117,245)]
def prawdopodobienstwo(res, gesty, klatka_wejsciowa, kolory):
    klatka_wyjsciowa = klatka_wejsciowa.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(klatka_wyjsciowa, (0,60+num*40), ((prob*100).astype(int), 90+num*40), kolory[num], -1)
        cv2.putText(klatka_wyjsciowa, gesty[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return klatka_wyjsciowa

In [22]:
sekwencja = []
zdanie = []
prog = 0.8

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        wartosc, klatka = cap.read()

        obraz, wyniki = rozpoznawanie(klatka, holistic)
        
        szkielet(obraz, wyniki)
        
        keypoints = zwroc_punkty(wyniki)
        
        sekwencja.append(keypoints)
        sekwencja = sekwencja[-30:]
        
        if len(sekwencja) == 30:
            res = model.predict(np.expand_dims(sekwencja, axis=0))[0]
            print(gesty[np.argmax(res)])
            
            
            if res[np.argmax(res)] > prog: 
                if len(zdanie) > 0: 
                    if gesty[np.argmax(res)] != zdanie[-1]:
                        zdanie.append(gesty[np.argmax(res)])
                else:
                    zdanie.append(gesty[np.argmax(res)])

            if len(zdanie) > 5: 
                zdanie = zdanie[-5:]

            obraz = prawdopodobienstwo(res, gesty, cv2.flip(obraz, 1), kolory)
            
        cv2.rectangle(obraz, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(obraz, ' '.join(zdanie), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('Jezyk migowy', obraz)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 208ms/step
czesc
1/1 [==============================] - 0s 35ms/step
czesc
1/1 [==============================] - 0s 37ms/step
czesc
1/1 [==============================] - 0s 41ms/step
czesc
1/1 [==============================] - 0s 37ms/step
czesc
1/1 [==============================] - 0s 40ms/step
czesc
1/1 [==============================] - 0s 43ms/step
czesc
1/1 [==============================] - 0s 34ms/step
czesc
1/1 [==============================] - 0s 35ms/step
czesc
1/1 [==============================] - 0s 36ms/step
czesc
1/1 [==============================] - 0s 35ms/step
czesc
1/1 [==============================] - 0s 38ms/step
czesc


In [23]:
cap.release()
cv2.destroyAllWindows()